# 📊 Processing Large CSV Files in Python


This notebook demonstrates how to handle large CSV/TSV files efficiently using Python. 
We will implement two key functionalities:
1. **Batch Processing with Column Removal**: Load and process data in chunks to save memory, and optionally remove unnecessary columns.
2. **Sampling Data**: Extract a specific percentage of rows, either sequentially or randomly, for quick analysis.

Both methods ensure memory efficiency while handling datasets that may not fit into memory.


In [1]:
# Importing necessary libraries
import pandas as pd
import os
import glob
import numpy as np
from tqdm import tqdm

## Batch Processing Large CSV Files


This function processes a large CSV/TSV file in smaller batches, removes unnecessary columns, 
and combines the processed data into a final output file. Temporary files are created during processing 
and are deleted after the final result is saved.


In [2]:
def process_large_csv_in_batches(
    file_path, 
    columns_to_drop, 
    batch_size=10000, 
    output_folder='temp_batches', 
    final_output='final_data.pkl',
    sep='	'
):
    """
    Processes a large CSV file in batches.

    Parameters:
    - file_path (str): Path to the input CSV/TSV file.
    - columns_to_drop (list): List of column names to drop from each batch.
    - batch_size (int): Number of rows to process in each batch.
    - output_folder (str): Temporary folder to store processed batches.
    - final_output (str): Path to save the final combined output.
    - sep (str): Delimiter used in the CSV/TSV file.
    """
    # Create a temporary folder for batch storage
    os.makedirs(output_folder, exist_ok=True)

    # Initialize batch counter
    batch_count = 0
    
    # Load the file in chunks
    for chunk in tqdm(pd.read_csv(file_path, sep=sep, chunksize=batch_size), desc="Processing Batches"):
        # Remove specified columns
        chunk = chunk.drop(columns=columns_to_drop, errors='ignore')
        
        # Save the processed batch to a pickle file
        batch_file = os.path.join(output_folder, f'batch_{batch_count}.pkl')
        chunk.to_pickle(batch_file)
        
        # Clear memory
        del chunk
        
        # Increment batch counter
        batch_count += 1
        print(f"Batch {batch_count} processed and saved.")

    # Combine all batch files into a single DataFrame
    all_batches = glob.glob(os.path.join(output_folder, 'batch_*.pkl'))
    final_df = pd.concat([pd.read_pickle(batch) for batch in all_batches], ignore_index=True)
    
    # Save the combined result to a single file
    final_df.to_pickle(final_output)
    print(f"Final data saved to {final_output}")
    
    # Clear memory
    del final_df
    
    # Clean up temporary files
    for batch in all_batches:
        os.remove(batch)
    os.rmdir(output_folder)
    print("Temporary files deleted.")

In [3]:
# Example usage for batch processing
process_large_csv_in_batches(
    file_path='/kaggle/input/vk-salary-train/train.tsv',  # Update path as needed
    columns_to_drop=['raw_description', 'raw_branded_description'],
    batch_size=10000,
    output_folder='temp_batches',
    final_output='final_data.pkl',
    sep='\t'
)

Processing Batches: 1it [00:01,  1.67s/it]

Batch 1 processed and saved.


Processing Batches: 2it [00:03,  1.54s/it]

Batch 2 processed and saved.


Processing Batches: 3it [00:04,  1.49s/it]

...

Processing Batches: 5it [00:07,  1.45s/it]

Batch 5 processed and saved.


Processing Batches: 7it [00:08,  1.28s/it]

Batch 6 processed and saved.
Batch 7 processed and saved.


Final data saved to final_data.pkl
Temporary files deleted.


## Sampling Data from a Large CSV File


This function samples a subset of rows from a large CSV/TSV file. The sampling can be:
- **Sequential**: Extract the first N% of rows.
- **Random**: Select a random N% of rows without loading the entire file into memory.

The sampled data is saved into a separate CSV file.


In [4]:
def sample_csv(
    file_path, 
    percentage=10, 
    random_sample=False, 
    output_file='sampled_data.csv',
    sep=','
):
    """
    Samples a specific percentage of data from a large CSV file.

    Parameters:
    - file_path (str): Path to the input CSV/TSV file.
    - percentage (int): Percentage of rows to sample (1-100).
    - random_sample (bool): If True, randomly sample rows; otherwise, sample sequentially.
    - output_file (str): Path to save the sampled data.
    - sep (str): Delimiter used in the CSV/TSV file.
    """
    if percentage <= 0 or percentage > 100:
        raise ValueError("Percentage must be between 1 and 100.")
    
    # Calculate the total number of rows
    total_rows = sum(1 for _ in open(file_path)) - 1  # Subtract 1 for the header
    sample_size = int(total_rows * (percentage / 100))
    
    if random_sample:
        # Randomly sample rows without reading the entire file
        skip_rows = sorted(np.random.choice(range(1, total_rows + 1), total_rows - sample_size, replace=False))
        sampled_df = pd.read_csv(file_path, sep=sep, skiprows=skip_rows)
    else:
        # Sequentially sample the first N% rows
        sampled_df = pd.read_csv(file_path, sep=sep, nrows=sample_size)
    
    # Save the sampled data to a new CSV file
    sampled_df.to_csv(output_file, index=False)
    print(f"Sampled data saved to {output_file}")

In [5]:
# Example usage for sampling
sample_csv(
    file_path='/kaggle/input/vk-salary-train/train.tsv',  # Update path as needed
    percentage=10,
    random_sample=True,
    output_file='sampled_data.csv',
    sep='\t'
)

Sampled data saved to sampled_data.csv
